# Agents Prompt Modification

In this notebook, we demonstrate an agent that changes the prompt of another agent. This would constitute a kind of quality intelligence improvement.

In [ ]:
# an evaluator agent evaluates the performance of a task agent
# the evaluator agent stores a prompt for a task agent in a file
# the task agent performs a task and stores a transcript of its actions to a file
# the evaluator reads the transcript and evaluates the task agent's performance
# the evaluator decides how to change the task agent's prompt based on the performance
# the task agent reads the new prompt and performs the task again

We're going to work on the 20_newsgroups dataset for classification.

## Evaluator Agent

In [15]:
from langchain import SerpAPIWrapper
from langchain.agents import AgentExecutor, initialize_agent, AgentType, OpenAIFunctionsAgent
from langchain.tools import BaseTool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
import os
from dotenv import load_dotenv
from typing import Type

load_dotenv()

search = SerpAPIWrapper(serpapi_api_key=os.environ['SERPAPI_API_KEY'])
llm = ChatOpenAI(model='gpt-3.5-turbo-0613', openai_api_key=os.environ['OPENAI_API_KEY'])

def evaluate():
    pass

class EvaluateSingleTranscriptToolInput(BaseModel):
    task_agent_prompt: str = Field(description="The prompt used by the task agent to perform the task.")
    transcript: str = Field(description="The transcript of a task agent's actions.")

class EvaluateSingleTranscriptTool(BaseTool):
    """
    This evaluates a single transcript of a task agent's actions. It tries to assess whether
    the task agent's actions were successful or not. It could be a simple loss function, or
    it could be a more complex function that tries to determine the cause of the task agent's
    failure.
    """
    name = "Evaluate Transcript"
    description = "Evaluate the transcript of a task agent's actions."
    args_schema: Type[BaseTool] = EvaluateSingleTranscriptToolInput

    def _run(self, transcript: str, prmopt: str):
        pass # could be some sort of loss function?
    def _arun(self, transcript: str, prmopt: str):
        pass

class ModifyPromptToolInput(BaseModel):

    task_agent_prompt: str = Field(description="The prompt used by the task agent to perform the task.")
    evaluation: str = Field(description="The evaluation of the task agent's performance.")

class ModifyPromptTool(BaseTool):
    """
    Seems like it's going to be limited if it can only modify the prompt based on a single transcript.
    Probably the EvaluateTranscriptsTool needs to be able to evaluate multiple transcripts, then
    come up with an abstraction for a common cause as to why the task agent failed, then modify the prompt
    based on that abstraction.
    """
    name = "Modify Prompt"
    description = "Modify the prompt of a task agent based on its performance."
    args_schema: Type[BaseModel] = ModifyPromptToolInput
    def _run(self, evaluation: str, task_agent_prompt: str):
        prompt = PromptTemplate(input_variables=[
            'task_agent_prompt',
            'transcript',
            'evaluation'
        ], template="""
        You are an evaluator of AI agents. You review task agents' transcripts and decide how to change their prompts.
        The task agent's original prompt was: {task_agent_prompt}
        The task agent's transcript is: {transcript}
        The task agent's evaluation is: {evaluation}

        You decide to change the task agent's prompt to:
        """)
        chain = LLMChain(llm=llm, prompt=prompt)
        chain.run({'task_agent_prompt': task_agent_prompt, 'evaluation': evaluation})

    def _arun(self, evaluation: str, transcript: str, prmopt: str):
        pass

tools = [
    EvaluateSingleTranscriptTool(llm),
    ModifyPromptTool(llm)
]

evaluator_agent = OpenAIFunctionsAgent.from_llm_and_tools(
    llm=llm, 
    tools=tools,
    prefix="You are an evaluator of AI agents. You review task agents' transcripts and decide how to change their prompts."
    )

evaluator_agent_executor = AgentExecutor.from_agent_and_tools(evaluator_agent, tools)

SyntaxError: unterminated string literal (detected at line 42) (1901114429.py, line 42)

## Task Agent

In [ ]:
task_agent_prefix = open('./task_agent_prefix.txt', 'r').read()
task_agent = OpenAIFunctionsAgent.from_llm_and_tools(
    llm=llm,
    tools=tools,
    prefix=task_agent_prefix
    )